In [1]:
from numpy import mean, var
import random
from copy import deepcopy

In [2]:
import platform
print(platform.python_version())

3.7.4


In [6]:
class Drawers:
    def __init__(self, max_bound):
        drawers = [i for i in range(1, max_bound + 1)]
        self._list = drawers
    
    def __getitem__(self, one_index):
        zero_index = one_index - 1
        return self._list[zero_index]
    
    def empty(self, one_index):
        zero_index = one_index - 1
        value = self._list[zero_index]
        self._list[zero_index] = None
        return value

In [7]:
class Bank:
    def __init__(self, drawers_count):
        self.drawers = Drawers(drawers_count)
    
    def ispresent(self, drawer):
        return self.drawers[drawer] is not None
        
    def withdraw(self, drawer):
        return self.drawers.empty(drawer)

In [9]:
class BaseStrategy:
    def __init__(self, bank):
        self.bank = deepcopy(bank)
        self.original_bank = deepcopy(bank)
    
    def restart(self):
        self.bank = deepcopy(self.original_bank)

In [11]:
class PairsFirstStrategy:
    def __init__(self, bank):
        self.base_strategy = BaseStrategy(bank)
    
    def __getattr__(self, attr):
        return getattr(self.base_strategy, attr)
    
    def execute(self, draw1, draw2):
        draw_sum = draw1 + draw2
        if self.bank.ispresent(draw1) and self.bank.ispresent(draw2):
            return self.bank.withdraw(draw1) + self.bank.withdraw(draw2)
        elif self.bank.ispresent(draw_sum):
            return self.bank.withdraw(draw_sum)
        else:
            return None

In [13]:
class PairsLastStrategy:
    def __init__(self, bank):
        self.base_strategy = BaseStrategy(bank)
    
    def __getattr__(self, attr):
        return getattr(self.base_strategy, attr)
    
    def execute(self, draw1, draw2):
        draw_sum = draw1 + draw2
        if self.bank.ispresent(draw_sum):
            return self.bank.withdraw(draw_sum)
        elif self.bank.ispresent(draw1) and self.bank.ispresent(draw2):
            return self.bank.withdraw(draw1) + self.bank.withdraw(draw2)
        else:
            return None

In [14]:
class RandomDrawStrategy:
    def __init__(self, bank):
        self.base_strategy = BaseStrategy(bank)
    
    def __getattr__(self, attr):
        return getattr(self.base_strategy, attr)
    
    def execute(self, draw1, draw2):
        draw_sum = draw1 + draw2
        if self.bank.ispresent(draw_sum) and self.bank.ispresent(draw1) and self.bank.ispresent(draw2):
            if random.random() > 0.49:
                return self.bank.withdraw(draw_sum)
            else:
                return self.bank.withdraw(draw1) + self.bank.withdraw(draw2)
        elif self.bank.ispresent(draw_sum):
            return self.bank.withdraw(draw_sum)
        elif self.bank.ispresent(draw1) and self.bank.ispresent(draw2):
            return self.bank.withdraw(draw1) + self.bank.withdraw(draw2)
        else:
            return None

In [16]:
class EntangledDice:
    def __init__(self, total):
        self.total = total
    
    def roll(self):
        results = [random.randrange(1, 7) for _ in range(self.total)]
        if len(set(results)) == 1:
            return self.roll()
        else:
            return results

In [17]:
class Game:    
    def __init__(self, strategy):
        self.strategy = strategy
        self.dice = EntangledDice(2)
                
    def turn(self):
        dice1, dice2 = self.dice.roll()
        return self.strategy.execute(dice1, dice2)
    
    def play(self):
        self.strategy.restart()
        responses = []
        while True:
            response = self.turn()
            if response is None:
                return responses
            else:
                responses.append(response)
                

In [18]:
class MonteCarlo:
    def __init__(self, games, total_runs = 1_000):
        self.games = games
        self.total_runs = total_runs
        
    def run(self):
        results = []
        for game in self.games:
            game_results = []
            for _ in range(self.total_runs):
                game_results.append(game.play())
                
            results.append(game_results)
        return results


In [19]:
bank = Bank(12)
pairs_first_strategy = PairsFirstStrategy(bank)
pairs_last_strategy = PairsLastStrategy(bank)
random_draw_strategy = RandomDrawStrategy(bank)
pairs_first_game = Game(pairs_first_strategy)
pairs_last_game = Game(pairs_last_strategy)
random_draw_game = Game(random_draw_strategy)
monte_carlo = MonteCarlo([pairs_first_game, pairs_last_game, random_draw_game], 10_000)
outputs = monte_carlo.run()

In [20]:
expectations = []
for output in outputs:
    game_expectations = []
    for result in output:
        exp = mean(result)
        game_expectations.append(exp)
        
    expectations.append(game_expectations)

mean(expectations, axis = 1)

array([7.23810115, 6.97177179, 7.13101167])